# Do the same thing for the UDF_f105w.fits file as for the sep_tutorial

In [ ]:
import sep
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from astropy.utils.data import download_file
from matplotlib.patches import Ellipse
from astropy import visualization as vis
%matplotlib inline

In [ ]:
f105w = download_file('https://archive.stsci.edu/pub/hlsp/hudf12/hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits')
hdul = fits.open(f105w)
data = hdul[0].data
hdul.close()

In [ ]:
# I will define functions for everything to make the code as compressible as possible.
# This function will do a basic plot of the FITS file
def FITS_plotter(array,figsz,interpol,cmp,orig,save_file_name):
    m, s = np.mean(array), np.std(data)
    plt.figure(figsize=figsz)
    plt.imshow(array, interpolation=interpol, cmap=cmp, vmin=m-s, vmax=m+s, origin=orig)
    plt.colorbar()
    plt.savefig(save_file_name);

In [ ]:
FITS_plotter(data,(15,15),'nearest','gray','lower','f105w_direct.png')

## Background Subtraction and Object Detection

In [ ]:
data = data.byteswap(inplace=True).newbyteorder()

In [ ]:
# Create a function containing all of the data needed for the background
# subtraction and object detection.
def Object_Data():
    
    # Define the variables that will be used in a class later as global
    global v, bkg, bkg_back, bkg_rms, gback, grms, data_sub, adata_sub, aobjects, sxt, aflux, afluxerr, aflag
    
    # This will be used to provide for the m, s in the plots and for the background
    # and rms images.
    bkg = sep.Background(data, bw=16, bh=16)
    data_sub = data - bkg
    bkg_back = bkg.back()
    bkg_rms = bkg.rms()
    
    # The data will be turned into integer slices
    ndata = {}
    v = 0
    for i in range(36):
        for n in range(36):
            ndata[36*i + n] = data[100*i:100*i+100:1,100*n:100*n+100:1]
            v += 1
    
    # This line converts the dict to an array
    adata = np.array(list(ndata.values()))
    
    # This section will define the values for the global background, global rms, and the data
    # array with the background subtracted.
    ndata_sub = {}
    gback = {}
    grms = {}
    for i in range(v):
        ndata_sub[i] = adata[i] - sep.Background(adata[i], bw=32, bh=32)
        gback[i] = sep.Background(adata[i], bw=32, bh=32).globalback
        grms[i] = sep.Background(adata[i], bw=32, bh=32).globalrms
    adata_sub = np.array(list(ndata_sub.values()))

    # Convert the gback and grms dicts into arrays
    agback = np.array(list(gback.values()))
    agrms = np.array(list(grms.values()))
    
    # Take the average of all of the relevant grms values
    u = 0
    for i in range(v):
        if(agrms[i]==1):
            u += 1
        else:
            continue
    Avg_agrms = (sum(agrms)-u)/(v-u)
                    
    # Place the raw sep.extract data into the sxt dict and the len(sep.extract) data into an objects dict
    objects = {}
    sxt = {}
    for i in range(v):
        try:
            p = np.exp(agrms[i]/Avg_agrms)
        except:
            p = 1
        q = sep.extract(adata_sub[i], p, err=sep.Background(adata[i], bw=32, bh=32).globalrms)
        sxt[i] = sep.extract(adata_sub[i], p, err=sep.Background(adata[i], bw=32, bh=32).globalrms)
        if(len(q)>30):
            q = sep.extract(adata_sub[i], 30*p, err=sep.Background(adata[i], bw=32, bh=32).globalrms)
        objects[i] = len(q)
    aobjects = np.array(list(objects.values()))
    
    dflux = {}
    dfluxerr = {}
    dflag = {}

    for n in range(1296):
        dflux[n], dfluxerr[n], dflag[n] = sep.sum_circle(adata_sub[n],sxt[n]['x'],sxt[n]['y'],np.exp(agrms[n]/Avg_agrms)\
                                                     ,err=agrms[n], gain=1.0)

    aflux = np.array(list(dflux.values()))
    afluxerr = np.array(list(dfluxerr.values()))
    aflag = np.array(list(dflag.values()))
    # Return the arrays and values
    return v, gback, grms, data_sub, adata_sub, aobjects, sxt, aflux, afluxerr, aflag

In [ ]:
Object_Data()

In [ ]:
# Now to define a class containing all of the functions that will be used for object
# detection.
class Object_Detection():
    def printer():
        for i in range(v):
            print('i:',i)
            print('\t','Global background:',gback[i])
            print('\t','Global rms:       ',grms[i])
            
    # Since plot_back and plot_rms don't need to be sliced, bkg.back and bkg.rms will
    # be plotted.
    def plot_back(figsz,interpol,cmp,orig,save_file_name):
        plt.figure(figsize=figsz)
        plt.imshow(bkg_back, interpolation=interpol, cmap=cmp, origin=orig)
        plt.colorbar()
        plt.savefig(save_file_name)
                      
    def plot_rms(figsz,interpol,cmp,orig,save_file_name):
        plt.figure(figsize=figsz)
        plt.imshow(bkg_rms, interpolation=interpol, cmap=cmp, origin=orig)
        plt.colorbar()
        plt.savefig(save_file_name)
        
    def object_detector(figsz,interpol,cmp,orig,save_file_name):
        im = {}
        fig, ax = plt.subplots(ncols=36, nrows=36, figsize=figsz)
        fig.subplots_adjust(hspace=0, wspace=0)
        w = 0
        z = 0
        m, s = np.mean(data_sub), np.std(data_sub)
        for n in range(v):
            im[n] = ax[36-z-1,w].imshow(adata_sub[n], interpolation=interpol, cmap=cmp, vmin=m-s, vmax=m+s, origin=orig)
            ax[36-z-1,w].axis('off')
    
            if(aobjects[n]>=25):
                if(w==36-1):
                    w = 0
                    z += 1
                else:
                    w += 1
            else:
                for i in range(aobjects[n]):
                    try:
                        if(aflux[n][i]<=0.05):
                            continue
                        else:
                            try:
                                e = Ellipse(xy=(sxt[n]['x'][i], sxt[n]['y'][i]),width=6*sxt[n]['a'][i],height=6*sxt[n]['b'][i],\
                                            angle=sxt[n]['theta'][i]*180/(2*np.pi))
                            except:
                                continue
                            e.set_facecolor('none')
                            e.set_edgecolor('red')
                            ax[35-z,w].add_artist(e)
                    except:
                        try:
                            e = Ellipse(xy=(sxt[n]['x'][i], sxt[n]['y'][i]),width=6*sxt[n]['a'][i],height=6*sxt[n]['b'][i],\
                                angle=sxt[n]['theta'][i]*180/(2*np.pi))
                        except:
                            continue
                        e.set_facecolor('none')
                        e.set_edgecolor('red')
                        ax[35-z,w].add_artist(e)
                if(w==36-1):
                    w = 0
                    z += 1
                else:
                    w += 1
        plt.savefig(save_file_name)

In [ ]:
# Now that the class is defined, lets start with the printer function.
Object_Detection.printer()

In [ ]:
# Now for the background image
Object_Detection.plot_back((15,15),'nearest','gray','lower','f105w_back.png')

In [ ]:
Object_Detection.plot_rms((15,15),'nearest','gray','lower','f105w_rms.png')

In [ ]:
Object_Detection.object_detector((15,15),'nearest','gray','lower','f105w_final.png')

### There are 2526 objects in this image, with uncertainty due to high noise around the border, the same objects being detected multiple times on slice boundaries, and the inability to differentiate low noise from especially faint objects in certain regions.

## Aperature Photometry and HIstogram

In [ ]:
fluxes = np.concatenate(aflux)
fluxerrs = np.concatenate(afluxerr)
flags = np.concatenate(aflag)

In [ ]:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i,fluxes[i],fluxerrs[i]))

### There is a difference between the number of objects and the number of fluxes as shown below. This is likely due to the detection of an object multiple times along the boundaries of the slices.

In [ ]:
print('Fluxes: ',len(fluxes))
print('Objects:',np.sum(aobjects))

In [ ]:
# The main plot is all of the fluxes, while the image at the top right is a magnification of the bottom left corner.
# This is done to give a better view of the low flux values that are otherwise unseen due to the high fluxes.
ax1 = plt.figure(figsize=(15,15)), plt.axes()
ax1 = vis.hist(fluxes,5092,range=(0,5))
ax2 = plt.axes([0.55, 0.55, 0.3, 0.3])
ax2 = vis.hist(fluxes,5092,range=(0,0.5))